#  Credit Card Fraud Detection

El objetivo de este análisis es predecir el fraude con tarjeta de crédito en los datos transaccionales. Utilizaré tensorflow para construir el modelo predictivo y t-SNE para visualizar el conjunto de datos en dos dimensiones al final de este análisis. 
Las secciones de este análisis incluyen:

  - Explorando los datos
  - Construyendo la red neuronal
  
Referencia: https://www.kaggle.com/currie32/predicting-fraud-with-tensorflow 

In [ ]:
import pandas as pd
import numpy as np 
import tensorflow as tf
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.gridspec as gridspec
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
# from show_confusion_matrix import show_confusion_matrix 

In [ ]:
df = pd.read_csv("../input/creditcard.csv")

## Explorando los datos

In [ ]:
df.head()

Los datos se transforman principalmente de su forma original, por razones de confidencialidad.

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

No hay valores perdidos, eso hace las cosas un poco más fáciles.

Veamos cómo se compara el tiempo en transacciones normales y fraudulentas.

In [ ]:
print ("Fraud")
print (df.Time[df.Class == 1].describe())
print ()
print ("Normal")
print (df.Time[df.Class == 0].describe())

In [ ]:
f, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(12,4))

bins = 50

ax1.hist(df.Time[df.Class == 1], bins = bins)
ax1.set_title('Fraud')

ax2.hist(df.Time[df.Class == 0], bins = bins)
ax2.set_title('Normal')

plt.xlabel('Time (in Seconds)')
plt.ylabel('Number of Transactions')
plt.show()

La función 'Tiempo' se ve bastante similar en ambos tipos de transacciones. Podría argumentar que las transacciones fraudulentas se distribuyen de manera más uniforme, mientras que las transacciones normales tienen una distribución cíclica. Esto podría facilitar la detección de una transacción fraudulenta durante el horario "fuera de temporada".

Ahora veamos si el monto de la transacción difiere entre los dos tipos.

In [ ]:
print ("Fraud")
print (df.Amount[df.Class == 1].describe())
print ()
print ("Normal")
print (df.Amount[df.Class == 0].describe())

In [ ]:
f, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(12,4))

bins = 30

ax1.hist(df.Amount[df.Class == 1], bins = bins)
ax1.set_title('Fraud')

ax2.hist(df.Amount[df.Class == 0], bins = bins)
ax2.set_title('Normal')

plt.xlabel('Amount ($)')
plt.ylabel('Number of Transactions')
plt.yscale('log')
plt.show()

In [ ]:
df['Amount_max_fraud'] = 1
df.loc[df.Amount <= 2125.87, 'Amount_max_fraud'] = 0

La mayoría de las transacciones son montos pequeños, menos de $ 100. Las transacciones fraudulentas tienen un valor máximo muy inferior a las transacciones normales, $ 2,125.87 frente a $ 25,691.16.

Comparemos el tiempo con la cantidad y veamos si podemos aprender algo nuevo.

In [ ]:
f, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(12,6))

ax1.scatter(df.Time[df.Class == 1], df.Amount[df.Class == 1])
ax1.set_title('Fraud')

ax2.scatter(df.Time[df.Class == 0], df.Amount[df.Class == 0])
ax2.set_title('Normal')

plt.xlabel('Time (in Seconds)')
plt.ylabel('Amount')
plt.show()

Nada demasiado útil aquí.

A continuación, echemos un vistazo a las características anónimas.

In [ ]:
# Seleccione solo las características anónimas.
v_features = df.ix[:,1:29].columns

In [ ]:
plt.figure(figsize=(12,28*4))
gs = gridspec.GridSpec(28, 1)
for i, cn in enumerate(df[v_features]):
    ax = plt.subplot(gs[i])
    sns.distplot(df[cn][df.Class == 1], bins=50)
    sns.distplot(df[cn][df.Class == 0], bins=50)
    ax.set_xlabel('')
    ax.set_title('histogram of feature: ' + str(cn))
plt.show()

In [ ]:
# Disminuya todas las características que tienen distribuciones muy similares entre los dos tipos de transacciones.
df = df.drop(['V28','V27','V26','V25','V24','V23','V22','V20','V15','V13','V8'], axis =1)

In [ ]:
# Basado en los trazados anteriores, estas características se crean para identificar valores donde las transacciones fraudulentas son más comunes.
df['V1_'] = df.V1.map(lambda x: 1 if x < -3 else 0)
df['V2_'] = df.V2.map(lambda x: 1 if x > 2.5 else 0)
df['V3_'] = df.V3.map(lambda x: 1 if x < -4 else 0)
df['V4_'] = df.V4.map(lambda x: 1 if x > 2.5 else 0)
df['V5_'] = df.V5.map(lambda x: 1 if x < -4.5 else 0)
df['V6_'] = df.V6.map(lambda x: 1 if x < -2.5 else 0)
df['V7_'] = df.V7.map(lambda x: 1 if x < -3 else 0)
df['V9_'] = df.V9.map(lambda x: 1 if x < -2 else 0)
df['V10_'] = df.V10.map(lambda x: 1 if x < -2.5 else 0)
df['V11_'] = df.V11.map(lambda x: 1 if x > 2 else 0)
df['V12_'] = df.V12.map(lambda x: 1 if x < -2 else 0)
df['V14_'] = df.V14.map(lambda x: 1 if x < -2.5 else 0)
df['V16_'] = df.V16.map(lambda x: 1 if x < -2 else 0)
df['V17_'] = df.V17.map(lambda x: 1 if x < -2 else 0)
df['V18_'] = df.V18.map(lambda x: 1 if x < -2 else 0)
df['V19_'] = df.V19.map(lambda x: 1 if x > 1.5 else 0)
df['V21_'] = df.V21.map(lambda x: 1 if x > 0.6 else 0)

In [ ]:
#Crea una nueva función para transacciones normales (no fraudulentas).
df.loc[df.Class == 0, 'Normal'] = 1
df.loc[df.Class == 1, 'Normal'] = 0

In [ ]:
# Renombrar 'Class' a 'Fraud'.
df = df.rename(columns={'Class': 'Fraud'})

In [ ]:
# 492 transacciones fraudulentas, 284,315 transacciones normales.
# 0.172% de las transacciones fueron fraude.
print(df.Normal.value_counts())
print()
print(df.Fraud.value_counts())

In [ ]:
pd.set_option("display.max_columns",101)
df.head()

In [ ]:
# Crear marcos de datos de solo Fraude y transacciones normales.
Fraud = df[df.Fraud == 1]
Normal = df[df.Normal == 1]

In [ ]:
# Establecer X_train igual al 80% de las transacciones fraudulentas.
X_train = Fraud.sample(frac=0.8)
count_Frauds = len(X_train)

# Agregar el 80% de las transacciones normales a X_train.
X_train = pd.concat([X_train, Normal.sample(frac = 0.8)], axis = 0)

# X_test contiene toda la transacción que no está en X_train.
X_test = df.loc[~df.index.isin(X_train.index)]

In [ ]:
# Deshaga los marcos de datos para que la capacitación se realice en un orden aleatorio.
X_train = shuffle(X_train)
X_test = shuffle(X_test)

In [ ]:
#Añadir nuestras características de destino a y_train y y_test.
y_train = X_train.Fraud
y_train = pd.concat([y_train, X_train.Normal], axis=1)

y_test = X_test.Fraud
y_test = pd.concat([y_test, X_test.Normal], axis=1)

In [ ]:
#Drop características de destino de X_train y X_test.
X_train = X_train.drop(['Fraud','Normal'], axis = 1)
X_test = X_test.drop(['Fraud','Normal'], axis = 1)

In [ ]:
#Verifique que todos los cuadros de datos de entrenamiento / prueba tengan la longitud correcta
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))

In [ ]:
Debido al desequilibrio en los datos, la relación actuará como un sistema de ponderación igual para nuestro modelo.
Al dividir el número de transacciones entre aquellas que son fraudulentas, la proporción será igual al valor que cuando se multiplique
por el número de transacciones fraudulentas será igual al número de transacciones normales.
En pocas palabras: # de fraude * ratio = # de normalidad


In [ ]:

ratio = len(X_train)/count_Frauds 

y_train.Fraud *= ratio
y_test.Fraud *= ratio

In [ ]:
#Nombres de todas las funciones en X_train.
features = X_train.columns.values

#Transforme cada característica en características para que tenga una media de 0 y una desviación estándar de 1;
# Esto ayuda a entrenar la red neuronal.
for feature in features:
    mean, std = df[feature].mean(), df[feature].std()
    X_train.loc[:, feature] = (X_train[feature] - mean) / std
    X_test.loc[:, feature] = (X_test[feature] - mean) / std

## Train the Neural Net

In [ ]:
# Dividir los datos de prueba en conjuntos de validación y prueba
split = int(len(y_test)/2)

inputX = X_train.as_matrix()
inputY = y_train.as_matrix()
inputX_valid = X_test.as_matrix()[:split]
inputY_valid = y_test.as_matrix()[:split]
inputX_test = X_test.as_matrix()[split:]
inputY_test = y_test.as_matrix()[split:]

In [ ]:
# Número de nodos de entrada.
input_nodes = 37

# Multiplier mantiene una proporción fija de nodos entre cada capa.
mulitplier = 1.5 

# Número de nodos en cada capa oculta
hidden_nodes1 = 18
hidden_nodes2 = round(hidden_nodes1 * mulitplier)
hidden_nodes3 = round(hidden_nodes2 * mulitplier)

# Porcentaje de nodos que mantener durante la deserción.
pkeep = tf.placeholder(tf.float32)

In [ ]:
# entrada
x = tf.placeholder(tf.float32, [None, input_nodes])

# layer 1
W1 = tf.Variable(tf.truncated_normal([input_nodes, hidden_nodes1], stddev = 0.15))
b1 = tf.Variable(tf.zeros([hidden_nodes1]))
y1 = tf.nn.sigmoid(tf.matmul(x, W1) + b1)

# layer 2
W2 = tf.Variable(tf.truncated_normal([hidden_nodes1, hidden_nodes2], stddev = 0.15))
b2 = tf.Variable(tf.zeros([hidden_nodes2]))
y2 = tf.nn.sigmoid(tf.matmul(y1, W2) + b2)

# layer 3
W3 = tf.Variable(tf.truncated_normal([hidden_nodes2, hidden_nodes3], stddev = 0.15)) 
b3 = tf.Variable(tf.zeros([hidden_nodes3]))
y3 = tf.nn.sigmoid(tf.matmul(y2, W3) + b3)
y3 = tf.nn.dropout(y3, pkeep)

# layer 4
W4 = tf.Variable(tf.truncated_normal([hidden_nodes3, 2], stddev = 0.15)) 
b4 = tf.Variable(tf.zeros([2]))
y4 = tf.nn.softmax(tf.matmul(y3, W4) + b4)

# salida
y = y4
y_ = tf.placeholder(tf.float32, [None, 2])

In [ ]:
# Parametros
training_epochs = 5 # debe ser 2000, se agotó el tiempo de espera al cargar
training_dropout = 0.9
display_step = 1 # 10 
n_samples = y_train.shape[0]
batch_size = 2048
learning_rate = 0.005

In [ ]:
# Función de costo: entropía cruzada
cost = -tf.reduce_sum(y_ * tf.log(y))

# Optimizaremos nuestro modelo a través de AdamOptimizer
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

# Corregir la predicción si el valor más probable (Fraude o Normal) de softmax es igual al valor objetivo.
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
accuracy_summary = []# Registrar valores de precisión para la trama
cost_summary = [] # Registrar los valores de costo para la trama
valid_accuracy_summary = [] 
valid_cost_summary = [] 
stop_early = 0 # Para realizar un seguimiento del número de épocas antes de detenerse temprano

# Guarda los mejores pesos para que puedan usarse para hacer las predicciones finales
#checkpoint = "location_on_your_computer / best_model.ckpt"
saver = tf.train.Saver(max_to_keep=1)

# Inicializar variables y sesión de tensorflow
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(training_epochs): 
        for batch in range(int(n_samples/batch_size)):
            batch_x = inputX[batch*batch_size : (1+batch)*batch_size]
            batch_y = inputY[batch*batch_size : (1+batch)*batch_size]

            sess.run([optimizer], feed_dict={x: batch_x, 
                                             y_: batch_y,
                                             pkeep: training_dropout})
        
        # Mostrar registros después de cada 10 épocas
        if (epoch) % display_step == 0:
            train_accuracy, newCost = sess.run([accuracy, cost], feed_dict={x: inputX, 
                                                                            y_: inputY,
                                                                            pkeep: training_dropout})

            valid_accuracy, valid_newCost = sess.run([accuracy, cost], feed_dict={x: inputX_valid, 
                                                                                  y_: inputY_valid,
                                                                                  pkeep: 1})

            print ("Epoch:", epoch,
                   "Acc =", "{:.5f}".format(train_accuracy), 
                   "Cost =", "{:.5f}".format(newCost),
                   "Valid_Acc =", "{:.5f}".format(valid_accuracy), 
                   "Valid_Cost = ", "{:.5f}".format(valid_newCost))
            
            # Guarde los pesos si se cumplen estas condiciones.
            #if epoch > 0 and valid_accuracy > max(valid_accuracy_summary) and valid_accuracy > 0.999:
            #    saver.save(sess, checkpoint)
            
            # Registra los resultados del modelo
            accuracy_summary.append(train_accuracy)
            cost_summary.append(newCost)
            valid_accuracy_summary.append(valid_accuracy)
            valid_cost_summary.append(valid_newCost)
            
            # Si el modelo no mejora después de 15 registros, detenga el entrenamiento.
            if valid_accuracy < max(valid_accuracy_summary) and epoch > 100:
                stop_early += 1
                if stop_early == 15:
                    break
            else:
                stop_early = 0
            
    print()
    print("Optimization Finished!")
    print()   
    
#with tf.Session() as sess:
    # Carga los mejores pesos y muestra sus resultados
    #saver.restore(sess, checkpoint)
    #training_accuracy = sess.run(accuracy, feed_dict={x: inputX, y_: inputY, pkeep: training_dropout})
    #validation_accuracy = sess.run(accuracy, feed_dict={x: inputX_valid, y_: inputY_valid, pkeep: 1})
    
    #print("Results using the best Valid_Acc:")
    #print()
    #print("Training Accuracy =", training_accuracy)
    #print("Validation Accuracy =", validation_accuracy)

In [ ]:
# Trazar la precisión y los resúmenes de costos
f, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(10,4))

ax1.plot(accuracy_summary) # blue
ax1.plot(valid_accuracy_summary) # green
ax1.set_title('Accuracy')

ax2.plot(cost_summary)
ax2.plot(valid_cost_summary)
ax2.set_title('Cost')

plt.xlabel('Epochs (x10)')
plt.show()

In [ ]:
# Encontrar los valores predichos, luego usarlos para construir una matriz de confusión
#predicted = tf.argmax(y, 1)
#with tf.Session() as sess:  
#    # Carga los mejores pesos
#    saver.restore(sess, checkpoint)
#    testing_predictions, testing_accuracy = sess.run([predicted, accuracy], 
#                                                     feed_dict={x: inputX_test, y_:inputY_test, pkeep: 1})
#    
#    print("F1-Score =", f1_score(inputY_test[:,1], testing_predictions))
#    print("Testing Accuracy =", testing_accuracy)
#    print()
#    c = confusion_matrix(inputY_test[:,1], testing_predictions)
#    show_confusion_matrix(c, ['Fraud', 'Normal'])

Aunque la red neuronal puede detectar la mayoría de las transacciones fraudulentas (82,93%), todavía hay algunas que se escaparon. Alrededor del 0,10% de las transacciones normales se clasificaron como fraudulentas, lo que lamentablemente puede sumar muy rápidamente dado el gran número de transacciones de tarjetas de crédito que ocurren cada minuto / hora / día. No obstante, estos modelos funcionan razonablemente bien y espero que si tuviéramos más datos, y si las características no se hubieran pretransformado, pudiéramos haber creado nuevas características y construido una red neuronal más útil.